### **Import libraries**

In [420]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from pandas.plotting import scatter_matrix
import pandas as pd
from sklearn.cluster import KMeans



In [ ]:
# Import your functions from fetch_data.py
from fetch_data import fetch_housing_data, load_housing_data
# Download and extract data if not done already
fetch_housing_data()
# Load the housing data into a pandas DataFrame
housing = load_housing_data()

### 🔍 **Take a Quick Look at the Data Structure**

Let’s begin by examining the top five rows using the DataFrame’s `head()` method.  
Each row in the dataset represents a district in California.


#### 📌 **There are 10 attributes in the dataset:**

- `longitude`  
- `latitude`  
- `housing_median_age`  
- `total_rooms`  
- `total_bedrooms`  
- `population`  
- `households`  
- `median_income`  
- `median_house_value`  
- `ocean_proximity`  


The `info()` method provides a quick summary of the dataset, including:
- The total number of entries (rows)
- Data types of each column
- The number of non-nul


In [ ]:
housing.head()

In [ ]:
housing.info()

In [ ]:
housing.describe()

In [ ]:
%matplotlib inline
# only in a Jupyter notebook

housing.hist(bins=50, figsize=(20,15))
plt.show()

### Notice a few things in these histograms:

1. **Median Income Scale**  
   The `median_income` attribute does not appear to be expressed in US dollars (USD).  
   After checking with the data collection team, you learn that:
   - The data has been **scaled** and **capped** at **15.0001** for high incomes and at **0.4999** for low incomes.
   - The values represent **tens of thousands of dollars** (e.g., `3` ≈ `$30,000`).

   > 📌 Working with preprocessed attributes is common in Machine Learning.  
   > It's not always an issue, but understanding how the data was processed is important.

2. **Capped Values: Housing Median Age and Median House Value**  
   - These features were also **capped**.
   - Capping the **median house value** may be a serious issue since it is the **target attribute** (label).
   - If the ML algorithm sees only values below a certain threshold (e.g., \$500,000), it may **learn incorrectly** that values never exceed this.

   #### What to do if precise predictions > \$500,000 are needed?
   - **Option 1**: Collect **uncapped labels** for those districts.
   - **Option 2**: Remove those districts from the **training and test sets** so the model is not unfairly penalized.

3. **Different Feature Scales**  
   - The features have very **different scales** (e.g., income vs. housing age).
   - This will require **feature scaling**, which we'll explore later in this chapter.

4. **Tail-Heavy Distributions**  
   - Many histograms are **right-skewed** (tail heavy).
   - This makes it harder for some ML algorithms to **detect patterns**.


### 🧪 **Create a Test Set**

It may seem strange to set aside part of the data before diving deeper into analysis.  
After all, shouldn’t we learn more about the data first?

However:

> ⚠️ **Your brain is an amazing pattern detection system, and that makes it prone to overfitting.**

If you examine the test set early, you may notice patterns and **subconsciously overfit** your model.  
This leads to **data snooping bias**: your system performs well in testing, but poorly in the real world.

#### ✅ **Best Practice:**

Create a test set **right away**:

- Pick a random subset of the data — typically **20%** (or less if the dataset is large)
- Set it aside and **never look at it** until the final evaluation stage



In [426]:
def split_train_data(data , test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices] , data.iloc[test_indices]

In [ ]:
train_set , test_set = split_train_data(housing , 0.2)
print(len(train_set))
print(len(test_set))

In [ ]:
train_set.head()

In [ ]:
test_set.head()

In [430]:
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

### 🎯 **Reliable & Representative Test Set Creation , Stratified Sampling**

Splitting the dataset randomly can cause the test set to have skewed proportions of key categories. Stratified sampling divides the data into subgroups (strata) and samples proportionally from each, preserving the overall distribution in the test set.

#### 🎯 **Why Stratified Sampling?**

Purely random sampling usually works well with large datasets. However, if the dataset is small or some features are unevenly distributed, random sampling may produce **biased samples**.

For example, in a survey of 1,000 people, the US population is roughly 51.3% female and 48.7% male. A well-designed survey maintains this ratio to get **representative results**. Pure random sampling could result in skewed gender ratios in the sample, biasing the outcomes.

This approach of dividing the population into **homogeneous subgroups (strata)** and sampling proportionally from each is called **stratified sampling**. It ensures the test set reflects the overall population accurately.


#### 🏷️ **Creating Income Categories for Stratification**

If a numerical feature like **median income** is crucial for prediction, you can turn it into a categorical attribute to stratify by income level.

Using `pd.cut()`, the continuous median income is split into 5 categories representing income ranges:

- Category 1: 0 to 1.5 (less than $15,000)  
- Category 2: 1.5 to 3  
- Category 3: 3 to 4.5  
- Category 4: 4.5 to 6  
- Category 5: 6 and above

This helps ensure the train and test sets maintain similar distributions of income levels.



#### 🧪 **Train-Test Split with Stratified Sampling**

To ensure the test set represents the overall population well—especially for imbalanced categories—we used **Stratified Sampling** via `Scikit-Learn`.

- The `ocean_proximity` feature is categorical and location-based, making it a good candidate for stratified sampling.
- To stratify on income, we created an `income_cat` attribute using the `median_income` feature.


In [431]:
housing['income_cat'] = pd.cut(housing['median_income'] , bins=[0.,1.5,3.0,4.5,6. , np.inf] , labels=[1,2,3,4,5])

In [ ]:
housing['income_cat'].hist()

In [433]:
split = StratifiedShuffleSplit(n_splits=1 , test_size=0.2 , random_state=42)
for train_index , test_index in split.split(housing , housing['income_cat']):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

- Let’s see if this worked as expected. You can start by looking at the income category
proportions in the test set:

In [ ]:
strat_test_set.income_cat.value_counts()/len(strat_test_set)

* We then performed a **Stratified Shuffle Split** to maintain the `income_cat` distribution in both training and test sets.

After the split, we removed the `income_cat` column to restore the datasets to their original state:


In [435]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [ ]:
housing.head()

### 🔍 Next Step: Data Exploration

With a properly split dataset, we now begin **exploring the data** to gain insights and prepare it for the machine learning pipeline.


In [437]:
housing = strat_train_set.copy()

### 📍 **Visualizing Geographical Data**

We have **geographical information**—specifically `latitude` and `longitude`. Plotting these as a scatterplot helps visualize **district locations** on the map.

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude")
plt.show()

In [ ]:
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)
plt.show()

### 🌍 **Visualizing Geographical Data with Housing Prices**

Now that we have a basic scatter plot of the districts using longitude and latitude, we can improve the visualization by adding more dimensions to the plot:

- **Circle radius** will represent the population of each district.
- **Color** will represent the median house value.
- We'll use the `jet` colormap (`cmap='jet'`) which ranges from blue (low) to red (high prices).

This enhanced scatter plot helps us spot geographic patterns in housing prices and population distribution more effectively.

In [ ]:
housing.plot(
    kind="scatter", 
    x="longitude", 
    y="latitude", 
    alpha=0.4,
    s=housing["population"] / 100,  # radius of circle
    label="Population",
    c="median_house_value",         # color represents house value
    cmap=plt.get_cmap("jet"),       # color map
    colorbar=True,
    figsize=(10,7)
)
plt.show()

### 🗺️ **Geographical Data Insights**

This image tells you that the housing prices are very much related to the **location** (e.g., close to the ocean) and to the **population density**, as you probably knew already.

It will probably be useful to:

- 🧠 Use a **clustering algorithm** to detect the main clusters.
- ➕ Add new features that measure the **proximity to the cluster centers**.
- 🌊 Consider the **ocean proximity** attribute, although in Northern California the housing prices in coastal districts are not always high — so it's not a simple rule.

These insights can guide better **feature engineering** and help improve your model's performance.


In [ ]:
# Extract latitude and longitude
housing_geo = housing[["latitude", "longitude"]]

# Apply KMeans clustering
kmeans = KMeans(n_clusters=5, random_state=42)
housing["cluster"] = kmeans.fit_predict(housing_geo)

# Add distance to cluster center as a new feature
housing["dist_to_center"] = np.linalg.norm(housing_geo - kmeans.cluster_centers_[housing["cluster"]], axis=1)

# Plot to visualize the clusters
plt.figure(figsize=(10, 7))
plt.scatter(housing["longitude"], housing["latitude"], c=housing["cluster"], cmap='rainbow', alpha=0.6, s=10)
plt.scatter(kmeans.cluster_centers_[:, 1], kmeans.cluster_centers_[:, 0], c='black', s=100, label="Centers")
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("KMeans Clustering of California Housing Districts")
plt.legend()
plt.grid(True)
plt.show()


### 🔍 **Looking for Correlations**

Since the dataset is not too large, we can compute the standard correlation coefficient (also called **Pearson’s r**) between every pair of attributes using the `corr()` method.

This allows us to see how strongly each feature is linearly related to the target variable `median_house_value`.

We will use the `.corr()` method on the DataFrame to get a correlation matrix and then inspect how each feature correlates with the target.


In [ ]:
# Compute the correlation matrix
corr_matrix = housing.corr(numeric_only=True)

# Display correlation of each feature with median_house_value
corr_matrix["median_house_value"].sort_values(ascending=False)

### **Understanding Correlation Coefficients**

The correlation coefficient ranges from **-1 to 1**:

- When it is close to **1**, it indicates a **strong positive correlation**. For example, the median house value tends to increase as median income increases.
- When it is close to **-1**, it indicates a **strong negative correlation**. For instance, there is a slight negative correlation between latitude and median house value, meaning prices tend to decrease as you move north.
- When it is close to **0**, it means **no linear correlation** exists between the variables.

### **Checking Correlations with Scatter Matrix**

Another useful way to explore correlations between attributes is to use Pandas' `scatter_matrix` function. This plots every numerical attribute against every other numerical attribute, making it easy to spot relationships.

Since our dataset has 11 numerical attributes (which would result in 121 plots), we will focus only on a few key attributes that appear most correlated with `median_house_value` for clarity.

These attributes are:  
- `median_house_value`  
- `median_income`  
- `total_rooms`  
- `housing_median_age`  

Below is the scatter matrix for these attributes.


In [ ]:
attributes = ["median_house_value", "median_income", "total_rooms",
"housing_median_age"]
scatter_matrix(housing[attributes] , figsize=(20,10))
plt.show()

### **Scatterplot: Median Income vs. Median House Value**

The main diagonal in the scatter matrix shows histograms of each attribute, which helps us understand their distributions.

Focusing on the most promising predictor — **median income** — we plot it against the **median house value**.

This scatterplot shows:

- A strong positive correlation: as median income increases, so does the median house value.
- Visible horizontal lines indicating price caps, notably at \$500,000 and other price thresholds around \$450,000, \$350,000, and \$280,000.
- These capped values represent quirks in the data that might mislead the model, so you may consider removing those districts during preprocessing.

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)
plt.show()

### **Experimenting with Attribute Combinations**

Hopefully, the previous sections gave you an idea of a few ways you can explore the data and gain insights. You identified a few data quirks that you may want to clean up before feeding the data to a Machine Learning algorithm, and you found interesting correlations between attributes, in particular with the target attribute.

You also noticed that some attributes have a tail-heavy distribution, so you may want to transform them (e.g., by computing their logarithm). Of course, your mileage will vary considerably with each project, but the general ideas are similar.

One last thing you may want to do before actually preparing the data for Machine Learning algorithms is to try out various attribute combinations. For example, the total number of rooms in a district is not very useful if you don’t know how many households there are. What you really want is the number of rooms per household.

Similarly, the total number of bedrooms by itself is not very useful: you probably want to compare it to the number of rooms. And the population per household also seems like a useful attribute to consider.


In [445]:
housing['rooms_per_household'] = housing['total_rooms'] / housing['households']
housing['rooms_per_household'] = housing['rooms_per_household'].round().astype(int)
housing['total_rooms'] = housing['total_rooms'].round().astype(int)
#housing['total_bedrooms'] = housing['total_bedrooms'].round().astype(int)
housing['population'] = housing['population'].round().astype(int)
housing['households'] = housing['households'].round().astype(int)

In [ ]:
housing.head()

In [ ]:
corr_matrix = housing.select_dtypes(include=[float, int]).corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
housing_encoded = pd.get_dummies(housing)
corr_matrix = housing_encoded.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

### **Preparing the Data for Machine Learning Algorithms**

This round of exploration does not have to be absolutely thorough; the point is to start off on the right foot and quickly gain insights that will help you get a first reasonably good prototype. 

This is an **iterative process**: once you get a prototype up and running, you can analyze its output to gain more insights and come back to this exploration step.

---

#### **Why Write Data Preparation Functions?**

Instead of preparing the data manually, it’s better to write **functions** for your transformations. This has several benefits:

- **Reproducibility**: Easily apply the same transformations to any dataset (e.g., when you get fresh data).
- **Reusability**: Build a library of transformation functions for future projects.
- **Integration**: Use the same functions in your live system to transform new data before prediction.
- **Experimentation**: Quickly try different transformation combinations to find the best setup.

---

#### **Next Step**

1. Revert to a clean training set (by copying `strat_train_set` again).
2. Separate **predictors** and **labels**, since transformations applied to features might not be the same as those applied to the target variable.

> **Note:** `drop()` creates a copy of the DataFrame and does **not** modify `strat_train_set` in place.


In [449]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

In [ ]:
housing_labels

### **Data Cleaning**

Most Machine Learning algorithms cannot work with missing features, so we need to handle them before training.

Earlier, we noticed that the **`total_bedrooms`** attribute has some missing values. There are three common strategies to fix this:

1. **Remove the corresponding rows** (districts) that have missing values.  
2. **Remove the entire column** that contains missing values.  
3. **Fill the missing values** with a specific value (e.g., 0, the mean, the median, etc.).



#### **Pandas Methods for Handling Missing Data**

- **`dropna()`** → Remove rows with missing values.  
- **`drop()`** → Remove specific columns.  
- **`fillna()`** → Replace missing values with a given constant or a computed value.



> The right choice depends on the dataset and problem — removing too much data can harm your model, while filling with the wrong value can introduce bias.


### **Handling Missing Values with Scikit-Learn**

If you choose **Option 3** (filling missing values with the median), remember:

- **Compute the median** only on the **training set**.
- **Save** the computed median so you can:
  - Fill missing values in the test set when evaluating.
  - Fill missing values in new incoming data once the system is live.

---

#### **Using `SimpleImputer` in Scikit-Learn**

Instead of manually computing and applying the median, Scikit-Learn provides the **`SimpleImputer`** class, which can automatically handle missing values.

In [451]:
imputer = SimpleImputer(strategy='median')
#Since the median can only be computed on numerical attributes, we need to create a copy of the data without the text attribute ocean_proximity:
housing_num = housing.drop('ocean_proximity' , axis=1)

In [ ]:
#Now you can fit the imputer instance to the training data using the fit() method:
imputer.fit(housing_num)

The `SimpleImputer` stores each attribute’s median in `statistics_`.  
Right now, only **`total_bedrooms`** has missing values, but future data may have gaps in other numeric fields.  
To be safe, apply the imputer to **all numeric attributes**.


In [ ]:
imputer.statistics_

In [ ]:
housing_num.median().values

Now we can use the **trained** imputer to replace missing values with the learned medians:

In [455]:
X = imputer.transform(housing_num)

The result is a NumPy array of transformed features.  
To convert it back to a DataFrame:

In [456]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns)

In [ ]:
housing_tr

### **Handling Text & Categorical Values.**
Most Machine Learning algorithms prefer to work with numbers anyway, so let’s con‐
vert these categories from text to numbers. For this, we can use Scikit-Learn’s Ordina
lEncoder class19:

In [ ]:
housing_cat = housing[["ocean_proximity"]]
housing_cat.head()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

### **Custom Transformers**

In [ ]:
housing_tr

In [461]:
from sklearn.base import BaseEstimator, TransformerMixin
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6
class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True): # no *args or **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]
        population_per_household = X[:, population_ix] / X[:, households_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
            bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In this example the transformer has one hyperparameter, add_bedrooms_per_room,
set to True by default (it is often helpful to provide sensible defaults). This hyperpara‐
meter will allow you to easily find out whether adding this attribute helps the
Machine Learning algorithms or not.

### **Feature Scaling**
One of the most important transformations you need to apply to your data is feature
scaling. With few exceptions, Machine Learning algorithms don’t perform well when
the input numerical attributes have very different scales.

### **Transformation Pipelines**
As you can see, there are many data transformation steps that need to be executed in
the right order. Fortunately, Scikit-Learn provides the Pipeline class to help with
such sequences of transformations. Here is a small pipeline for the numerical
attributes:

In [462]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="median")),
    ('attribs_adder', CombinedAttributesAdder()),
    ('std_scaler', StandardScaler()),
])

housing_num_tr = num_pipeline.fit_transform(housing_num)

So far, we have handled the categorical columns and the numerical columns sepa‐
rately. It would be more convenient to have a single transformer able to handle all col‐
umns, applying the appropriate transformations to each column. In version 0.20,
Scikit-Learn introduced the ColumnTransformer for this purpose, and the good news
is that it works great with Pandas DataFrames. Let’s use it to apply all the transforma‐
tions to the housing data:

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

num_attrbs = list(housing_num)
cat_attrbs = ['ocean_proximity']

full_pipeline = ColumnTransformer([
    ('nums' , num_pipeline , num_attrbs),
    ('cat' , OneHotEncoder() , cat_attrbs)
])

housing_prepared = full_pipeline.fit_transform(housing)
housing_prepared

### **Select and Train a Model**
At last! You framed the problem, you got the data and explored it, you sampled a
training set and a test set, and you wrote transformation pipelines to clean up and
prepare your data for Machine Learning algorithms automatically. You are now ready
to select and train a Machine Learning model.

### **Training and Evaluating on the Training Set**

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared , housing_labels)

In [ ]:
some_data = housing.iloc[:5]
some_labels = housing_labels.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print('Predictions: ' , lin_reg.predict(some_data_prepared))
print('Labels:' , some_labels)

It works, although the predictions are not exactly accurate (e.g., the first prediction is
off by close to 40%!). Let’s measure this regression model’s RMSE on the whole train‐
ing set using Scikit-Learn’s mean_squared_error function:

In [ ]:
from sklearn.metrics import mean_squared_error
housing_predictions = lin_reg.predict(housing_prepared)
lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)
print(f'Error: {lin_rmse}')

- Okay, this is better than nothing but clearly not a great score: most districts’
median_housing_values range between $120,000 and $265,000, so a typical predic‐
tion error of $68,628 is not very satisfying. This is an example of a model underfitting
the training data. When this happens it can mean that the features do not provide
enough information to make good predictions, or that the model is not powerful
enough. As we saw in the previous chapter, the main ways to fix underfitting are to
select a more powerful model, to feed the training algorithm with better features, or
to reduce the constraints on the model. This model is not regularized, so this rules
out the last option. You could try to add more features (e.g., the log of the popula‐
tion), but first let’s try a more complex model to see how it does.
Let’s train a DecisionTreeRegressor. 

- This is a powerful model, capable of finding
complex nonlinear relationships in the data (Decision Trees are presented in more
detail in Chapter 6). The code should look familiar by now:

In [ ]:
from sklearn.tree import DecisionTreeRegressor
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared, housing_labels)

#Now that the model is trained, let’s evaluate it on the training set:
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)
print(f"Error: {tree_rmse}")

Could this model really be absolutely perfect? Of course,
it is much more likely that the model has badly overfit the data. How can you be sure?
As we saw earlier, you don’t want to touch the test set until you are ready to launch a
model you are confident about, so you need to use part of the training set for train‐
ing, and part for model validation.